## Quantitative Momentum Strategy

### Low Quality Momentum Strategy

#### Import Libraries and Data

In [1]:
import numpy as np 
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.rename(columns={'Symbol': 'Ticker'}, inplace = True)
stocks.head()

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


#### Batch API Calls

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

symbol_strings

['MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN',
 'CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC',
 'F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HU

In [4]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,184.56,0.055067,N/A
1,AOS,90.04,0.599476,N/A
2,ABT,145.20,0.335809,N/A
3,ABBV,138.40,0.362270,N/A
4,ABMD,370.05,0.122435,N/A
...,...,...,...,...
500,YUM,143.33,0.290251,N/A
501,ZBRA,597.80,0.575910,N/A
502,ZBH,130.70,-0.158660,N/A
503,ZION,64.96,0.499354,N/A


#### Remove Low-Momentum Stocks

In [5]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,46.01,1.859883,N/A
1,MCHP,87.32,1.604971,N/A
2,MRO,16.81,1.490465,N/A
3,FTNT,368.60,1.488598,N/A
4,F,21.25,1.367915,N/A
5,NVDA,295.03,1.311527,N/A
6,MRNA,255.48,1.273204,N/A
7,FANG,108.35,1.249261,N/A
8,NUE,115.67,1.198515,N/A
9,IT,339.95,1.109578,N/A


#### Calculate Number of Shares to Buy

In [6]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000000


In [7]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Users/bryanyu1/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,46.01,1.859883,426
1,MCHP,87.32,1.604971,224
2,MRO,16.81,1.490465,1166
3,FTNT,368.60,1.488598,53
4,F,21.25,1.367915,922
5,NVDA,295.03,1.311527,66
6,MRNA,255.48,1.273204,76
7,FANG,108.35,1.249261,180
8,NUE,115.67,1.198515,169
9,IT,339.95,1.109578,57


### High Quality Momentum Strategy

#### Build DataFrame

In [8]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

#### Calculate Momentum Percentiles 

In [9]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,182.46,N/A,0.053422,0.186139,-0.082464,0.136634,-0.010245,0.257426,0.008275,0.281188,N/A
1,AOS,89.24,N/A,0.592105,0.855446,0.208346,0.770297,0.362393,0.974257,0.056365,0.657426,N/A
2,ABT,147.46,N/A,0.330248,0.562376,0.219287,0.794059,0.185334,0.815842,0.101997,0.859406,N/A
3,ABBV,135.60,N/A,0.364954,0.60198,0.245933,0.847525,0.276842,0.946535,0.166700,0.982178,N/A
4,ABMD,366.26,N/A,0.118003,0.261386,0.159716,0.669307,0.102895,0.607921,0.137063,0.960396,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,141.16,N/A,0.297473,0.516832,0.214786,0.784158,0.133854,0.693069,0.105214,0.873267,N/A
501,ZBRA,604.80,N/A,0.578379,0.845545,0.118424,0.578218,0.136975,0.70297,-0.016383,0.150495,N/A
502,ZBH,129.16,N/A,-0.159695,0.035644,-0.209409,0.027723,-0.144112,0.041584,0.038854,0.49505,N/A
503,ZION,63.26,N/A,0.507619,0.782178,0.218790,0.790099,-0.003787,0.277228,-0.027446,0.120792,N/A


#### Calculate High Quality Momentum (HQM) Score

The HQM Score is the arithmetic mean of the 4 momentum percentile scores. 

In [10]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,182.46,N/A,0.053422,0.186139,-0.082464,0.136634,-0.010245,0.257426,0.008275,0.281188,0.215347
1,AOS,89.24,N/A,0.592105,0.855446,0.208346,0.770297,0.362393,0.974257,0.056365,0.657426,0.814356
2,ABT,147.46,N/A,0.330248,0.562376,0.219287,0.794059,0.185334,0.815842,0.101997,0.859406,0.757921
3,ABBV,135.60,N/A,0.364954,0.60198,0.245933,0.847525,0.276842,0.946535,0.166700,0.982178,0.844554
4,ABMD,366.26,N/A,0.118003,0.261386,0.159716,0.669307,0.102895,0.607921,0.137063,0.960396,0.624752
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,141.16,N/A,0.297473,0.516832,0.214786,0.784158,0.133854,0.693069,0.105214,0.873267,0.716832
501,ZBRA,604.80,N/A,0.578379,0.845545,0.118424,0.578218,0.136975,0.70297,-0.016383,0.150495,0.569307
502,ZBH,129.16,N/A,-0.159695,0.035644,-0.209409,0.027723,-0.144112,0.041584,0.038854,0.49505,0.15
503,ZION,63.26,N/A,0.507619,0.782178,0.218790,0.790099,-0.003787,0.277228,-0.027446,0.120792,0.492574


#### Select 50 Best Momentum Stocks

In [11]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,146.86,N/A,1.010918,0.976238,0.585674,0.992079,0.687337,1.0,0.132883,0.948515,0.979208
1,CF,71.90,N/A,0.970966,0.972277,0.495711,0.984158,0.284676,0.950495,0.147711,0.974257,0.970297
2,EXR,228.88,N/A,1.053768,0.978218,0.402243,0.966337,0.329585,0.966337,0.119992,0.916832,0.956931
3,AVGO,689.45,N/A,0.598538,0.863366,0.416297,0.976238,0.380187,0.978218,0.187301,0.99802,0.95396
4,PFE,61.35,N/A,0.674216,0.918812,0.543892,0.988119,0.366259,0.976238,0.119011,0.910891,0.948515
5,ACN,420.60,N/A,0.643799,0.89901,0.410069,0.970297,0.290388,0.956436,0.136823,0.956436,0.945545
6,PLD,174.95,N/A,0.762919,0.948515,0.397080,0.964356,0.334495,0.970297,0.104712,0.871287,0.938614
7,AZO,2155.17,N/A,0.776407,0.950495,0.414403,0.974257,0.205509,0.851485,0.135226,0.954455,0.932673
8,DRE,67.70,N/A,0.706843,0.936634,0.383163,0.960396,0.352138,0.972277,0.098403,0.845545,0.928713
9,LYV,122.89,N/A,0.622346,0.883168,0.354547,0.952475,0.309652,0.960396,0.113449,0.89505,0.922772


#### Calculate Number of Shares to Buy

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000000


In [13]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

hqm_dataframe

/Users/bryanyu1/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ANET,146.86,133,1.010918,0.976238,0.585674,0.992079,0.687337,1.0,0.132883,0.948515,0.979208
1,CF,71.90,272,0.970966,0.972277,0.495711,0.984158,0.284676,0.950495,0.147711,0.974257,0.970297
2,EXR,228.88,85,1.053768,0.978218,0.402243,0.966337,0.329585,0.966337,0.119992,0.916832,0.956931
3,AVGO,689.45,28,0.598538,0.863366,0.416297,0.976238,0.380187,0.978218,0.187301,0.99802,0.95396
4,PFE,61.35,319,0.674216,0.918812,0.543892,0.988119,0.366259,0.976238,0.119011,0.910891,0.948515
5,ACN,420.60,46,0.643799,0.89901,0.410069,0.970297,0.290388,0.956436,0.136823,0.956436,0.945545
6,PLD,174.95,112,0.762919,0.948515,0.397080,0.964356,0.334495,0.970297,0.104712,0.871287,0.938614
7,AZO,2155.17,9,0.776407,0.950495,0.414403,0.974257,0.205509,0.851485,0.135226,0.954455,0.932673
8,DRE,67.70,289,0.706843,0.936634,0.383163,0.960396,0.352138,0.972277,0.098403,0.845545,0.928713
9,LYV,122.89,159,0.622346,0.883168,0.354547,0.952475,0.309652,0.960396,0.113449,0.89505,0.922772


### Export High Quality Momentum Strategy Results to Excel

In [14]:
writer = pd.ExcelWriter('high_quality_momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [16]:
writer.save()